# Processamento de Linguagem Natural (PLN) - Básico


SERPRO - SUPSD - DIVISÃO DE DESENVOLVIMENTO E SUSTENTAÇÃO DE PRODUTOS COGNITIVOS

__JORNADA IA__

## Parte 3: Exemplo - Classificação de Documentos com CNN

Embora as CNNs tenham sido usadas principalmente para tarefas de Visão Computacional, nada as impede de serem usadas em aplicativos de PLN. Uma dessas aplicações para as quais as CNNs têm sido usadas efetivamente é a classificação de sentenças e documentos. 

A classificação de documentos é uma das tarefas mais populares em PLN. A classificação de documentos é extremamente útil para qualquer pessoa que esteja lidando com coleções massivas de dados, como as de sites de notícias, editores, advogados e universidades. Portanto, é interessante ver como a aprendizagem dos vetores de palavras pode ser adaptada a uma tarefa do mundo real, como a classificação de documentos por meio da vetorização de documentos inteiros em vez de palavras.

O Objetivo deste exemplo é classificar documentos utilizando CNN.


## Dataset 

Para essa tarefa, usaremos um conjunto de arquivos de texto já categorizados. Estes são artigos de notícias da BBC. Todos os documentos desta coleção pertencem a uma das seguintes categorias: 

- Negócios
- Entretenimento
- Política
- Esportes
- Tecnologia

Um exemplo de documento da categoria Tecnologia:

'UK net users leading TV downloads British TV viewers lead the trend of illegally downloading US shows from the net, according to research. New episodes of 24, Desperate Housewives and Six Feet Under, appear on the web hours after they are shown in the US, said a report. Web tracking company Envisional said 18% of downloaders were from within the UK and that downloads of TV programmers had increased by 150% in the last year....'

Página para o [dataset](http://mlg.ucd.ie/datasets/bbc.html)

Usaremos a seguinte estratégia:  
 - Extrair os dados
 - Pré-processamento dos dados
 - Construir a Rede Neural CNN
 - Treinamento do modelo
 - Avaliar o modelo
 - Realizando Predições com dados de Teste
 - Visualizar as métricas de classificação e matriz e monfusão

In [1]:
#!pip install pandas
#!pip install keras
# Imports
import os
import time
import numpy as np 
import pandas as pd
import re
import operator
import tensorflow as tf
import tflearn
import nltk
import sys
import unicodedata

from keras.preprocessing.text import Tokenizer

from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


## Carregamento dos dados 

Iremos extrair os dados e carregá-los em um DataFrame, nas colunas "Category" e "Text". 

In [2]:
# Função para leitura dos dados
def read_data_into_pandas(path):
    col_names =  ['Category', 'Text', 'Filename']
    data  = pd.DataFrame(columns = col_names)    
    for root, _, files in os.walk(path):        
        for filename in files:
            if filename.endswith(".txt"):
                with open(os.path.join(root,filename),'r',encoding='latin-1') as f: 
                    text = f.read()
                    data.loc[len(data)] = [root.split('/')[-1],text,filename]
    return data

path = 'data/bbc-fulltext'
data = read_data_into_pandas(path)
data = data.sample(frac=1) #Shuffle
data.head()

,Category,Text,Filename
1361,sport,Henin-Hardenne beaten on comeback\n\nJustine H...,465.txt
150,business,US trade gap ballooned in October\n\nThe US tr...,429.txt
510,politics,Plan to give elderly care control\n\nElderly a...,388.txt
1101,tech,"Lifestyle 'governs mobile choice'\n\nFaster, b...",225.txt
583,politics,Kilroy launches 'Veritas' party\n\nEx-BBC chat...,111.txt


## Pré-processamento dos dados

A função de pré-processamento dos dados vai realizar as seguintes tarefas:
- Remoção de Pontuação
- Tokenização
- Remoção de Stop words
- Aplicação de Stemmer

In [3]:
# Estrutura para armazenar pontuações
tbl = dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))

# Função para remover pontuações das sentenças
def remove_punctuation(text):
    return text.translate(tbl)

def preprocessing(text):
        
    # Remover pontuação
    text= remove_punctuation(text)
    
    # Tokenização
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    
    # Coloca em minúscula
    tokens = [word.lower() for word in tokens]
    
    # Remove stopwords
    # Existe um pacote de stop words padrão para português -> stopwords.words('portuguese')
    stop = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop]
    
    # Realizar o Stem
    stemmer = LancasterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    
    # Lemmatizer: opção em relação ao stem 
    # wordnet_lemmatizer = WordNetLemmatizer()
    # tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
    
    # Retonar o texto preprocessado agrupando as tokens
    preprocessed_text= ' '.join(tokens)
    return preprocessed_text

start = time.time()
data['Text'] = data['Text'].apply(preprocessing)
print(data['Text'][0])
print('Tempo decorrido %.3f' % (time.time()-start))
data.head()

venezuel ident idl farm venezuel auth ident 500 farm includ 56 larg est idl continu controvers land reform policy 2001 land law govern tax seiz unus farm sit 40000 farm yet inspect stat nat land institut told assocy press vic presid jos vic rangel said farm ranch titl ord land produc noth fear crit land reform policy claim presid hugo chavez try enforc communiststyl econom program ign property right dam country land own claim nat land institut mad mistak class land publ priv govern venezuela largest land own say process cauty prev conflict stat mr rangel said land reform constitut permit priv property stressing effort vind soc econom year ineq country on property conflict govern el charcot cattl ranch run agroflor subsidy uk food group vestey agricult min arnoldo marquez told reut new ag sit docu guar priv land admin ranch howev complain prochavez squat tak 80 property last four year uk govern ask venezuel auth resolv conflict ask company going put pap ord hand land said mr marquez
Tem

,Category,Text,Filename
1361,sport,heninharden beat comeback justin heninharden l...,465.txt
150,business,us trad gap balloon octob us trad deficit wid ...,429.txt
510,politics,plan giv eld car control eld dis peopl would c...,388.txt
1101,tech,lifestyl govern mobl cho fast bet funky hardw ...,225.txt
583,politics,kilroy launch verita party exbbc chat show hos...,111.txt


In [4]:
# Visualizando os dados após o processamento
data.loc[[0]]

,Category,Text,Filename
0,business,venezuel ident idl farm venezuel auth ident 50...,388.txt


## Vetorização dos Dados para o Treinamento

Utilizaremos a classe Tokenizer do __[Keras](https://keras.io/)__ para montar nosso vocabulário. Ela permite vetorizar o texto do corpus, transformando o texto na sequência de inteiros (onde cada inteiro representa o índice do token em um dicionário) ou em uma matriz de documentos com as contagem das palavras ou seus respectivos scores TF-IDF

https://keras.io/preprocessing/text/

Vetorização das variáveis preditoras (X)

In [5]:
# Criação do Vocabulário com base nos textos dos documentos
# Número de features (palavras) utilizadas para o treinamento
NUM_FEATURES=1000
tokenizer = Tokenizer(num_words=NUM_FEATURES)
tokenizer.fit_on_texts(data['Text'].values)
print("Tamanho do Vocabulário: ", len(tokenizer.word_index))
print("Top primeiras palavras do vocabulário: ")
iterator = iter(tokenizer.word_index.items())
top_words = [next(iterator) for i in range(20)]
print(top_words)

# Vetorização dos dados de treinamento
X = tokenizer.texts_to_sequences(data['Text'].values)
print("Exemplo de documento vetorizado:", X[0][:50])
#print("Exemplo de documento vetorizado:", X[0])


# Padding dos dados de treinamento: preencher com zeros para que todas as frases fiquem do mesmo tamanho
# Necessário para alimentar a rede neural  
X = pad_sequences(X,value=0.)

print("Exemplo de documento vetorizado preenchido com zeros): ", X[0][:50])
print("Tamanho do maior documento:",X.shape[1])
print("Shape do vetor de treinamento:",X.shape)

Tamanho do Vocabulário:  21734
Top primeiras palavras do vocabulário: 
[('said', 1), ('us', 2), ('mr', 3), ('year', 4), ('would', 5), ('new', 6), ('also', 7), ('peopl', 8), ('play', 9), ('on', 10), ('gam', 11), ('say', 12), ('tim', 13), ('could', 14), ('mak', 15), ('off', 16), ('last', 17), ('tak', 18), ('first', 19), ('govern', 20)]
Exemplo de documento vetorizado: [319, 328, 258, 430, 93, 356, 28, 90, 675, 24, 970, 144, 744, 703, 35, 305, 356, 72, 288, 340, 276, 357, 367, 199, 1, 85, 341, 292, 17, 13, 30, 200, 555, 781, 24, 50, 149, 430, 344, 90, 508, 200, 390, 248, 324, 99, 68, 1, 736, 27]
Exemplo de documento vetorizado preenchido com zeros):  [319 328 258 430  93 356  28  90 675  24 970 144 744 703  35 305 356  72
 288 340 276 357 367 199   1  85 341 292  17  13  30 200 555 781  24  50
 149 430 344  90 508 200 390 248 324  99  68   1 736  27]
Tamanho do maior documento: 1455
Shape do vetor de treinamento: (2225, 1455)


In [6]:
# Imprimindo o primeiro exemplo
X[0]

array([319, 328, 258, ...,   0,   0,   0], dtype=int32)

In [7]:
# Caso queira utilizar uma matriz podemos criar uma Bag of Word com TF-IFD das palavras para cada documento
X_matrix_tfidf = tokenizer.texts_to_matrix(data['Text'].values,mode='tfidf')
print("Shape Matrix TF-IDF",X_matrix_tfidf.shape)
print("Exemplo TF-IDF",X_matrix_tfidf[0])

Shape Matrix TF-IDF (2225, 1000)
Exemplo TF-IDF [0.         1.6334511  1.00288827 0.         0.98648218 0.
 0.         0.         0.         0.         0.         0.
 0.         2.56357913 0.         1.2649335  1.31642468 1.22689338
 0.         0.         0.         0.         1.47504034 0.
 3.07365032 0.         0.         2.43971249 1.31642468 0.
 1.42888369 0.         0.         0.         0.         3.14297232
 0.         0.         0.         0.         0.         0.
 0.         1.47504034 0.         0.         0.         0.
 0.         0.         1.62140976 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.57749977 0.         0.         0.
 1.62729228 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         3.06259006 0.         0.         0.         0.
 2.95101388 0.         0.         4.26675145 0.    

In [8]:
# Ou então uma Bag of Word com a contagem das palavras para cada documento
X_matrix_count = tokenizer.texts_to_matrix(data['Text'].values,mode='count')
print("Shape Matrix Count",X_matrix_count.shape)
print("Exemplo Frequencia",X_matrix_count[0])

Shape Matrix Count (2225, 1000)
Exemplo Frequencia [0. 3. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0.
 3. 0. 0. 2. 1. 0. 1. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 2. 0. 0. 4. 0. 0.
 0. 1. 0. 2. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0.


Vetorização da variável alvo (Y), que são as categorias dos documentos

In [9]:
#Vetorização dos labels (Target)
Y_labels = pd.get_dummies(data['Category'])
print("Categorias: ",Y_labels.columns)
Y = pd.get_dummies(data['Category']).values
print("Exemplo vetor de label:",Y[0])
print('Shape do vetor Y',Y.shape)

Categorias:  Index(['business', 'entertainment', 'politics', 'sport', 'tech'], dtype='object')
Exemplo vetor de label: [0 0 0 1 0]
Shape do vetor Y (2225, 5)


## Separação dos dados

Vamos separar os dados de treinamento (90%) e teste (10%). Para isso vamos usar a função __train_test_split()__ do scikit-learn

In [10]:
# Separação dos dados de treino e teste

# Para uso de X descomente a linha abaixo
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 42)

# Para uso de X_matrix_tfidf descomente a linha abaixo
#X_train, X_test, Y_train, Y_test = train_test_split(X_matrix_tfidf,Y, test_size = 0.2, random_state = 42)

# Para uso de X_matrix_count descomente a linha abaixo
#X_train, X_test, Y_train, Y_test = train_test_split(X_matrix_count,Y, test_size = 0.2, random_state = 42)

print('Dados de treino:',X_train.shape,Y_train.shape)
print('Dados de teste: ',X_test.shape,Y_test.shape)
print('Exemplo de dado de treino', X_train[0])

Dados de treino: (2002, 1455) (2002, 5)
Dados de teste:  (223, 1455) (223, 5)
Exemplo de dado de treino [ 14 369 709 ...   0   0   0]


## Construção da Rede Neural Convolucional

http://tflearn.org/layers/conv/#convolution-1d

Vamos criar uma rede neural convolucional com a seguinte arquitetura:
- O input será uma matriz com dimensão X (maior número de palavras de um documento, caso esteja usando a vetorização)
- Camada de entrada terá o número de neurônio igual ao número de features (palavras) do dicionário
- Três camadas convolucionais (1d por se tratar de um vetor) e função de ativação ReLU. As camadas terão 128 filtros e com tamanhos 3,5 e 7 respectivamente.
- Camada de Max Polling para realizar o agrupamento e redução da dimensionalidade
- Camada Dropout para evitar o overfitting
- Uma camada totalmente conectada com a função de ativação Softmax com as probabilidades das classes
- Por fim uma camada de regressão com a função de perda e otimização

In [11]:
# Reset do grafo tenforflow
tf.reset_default_graph()

# Building convolutional network

# Para uso de X descomente a linha abaixo
network = input_data(shape=[None, X.shape[1]], name='input')

# Para uso de X_matrix_tfidf descomente a linha abaixo
#network = input_data(shape=[None, X_matrix_tfidf.shape[1]], name='input')

# Para uso de X_matrix_count descomente a linha abaixo
#network = input_data(shape=[None, X_matrix_count.shape[1]], name='input')

network = tflearn.embedding(network, input_dim=NUM_FEATURES, output_dim=128)
branch1 = conv_1d(network, 128, 3, padding='same', activation='relu', regularizer="L2")
branch2 = conv_1d(network, 128, 5, padding='same', activation='relu', regularizer="L2")
branch3 = conv_1d(network, 128, 7, padding='same', activation='relu', regularizer="L2")
network = merge([branch1, branch2, branch3], mode='concat', axis=1)
network = tf.expand_dims(network, 2)
network = global_max_pool(network)
network = dropout(network, 0.5)
network = fully_connected(network, 5, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.001,
                     loss='categorical_crossentropy', name='target')
model = tflearn.DNN(network, tensorboard_verbose=0)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


## Treinamento do Modelo

In [12]:
model.fit(X_train, Y_train, n_epoch = 5,validation_set=0.1,batch_size=32,shuffle=True, show_metric=True)

Training Step: 284  | total loss: 0.15640 | time: 27.245s
| Adam | epoch: 005 | loss: 0.15640 - acc: 0.9549 -- iter: 1792/1801
Training Step: 285  | total loss: 0.14793 | time: 28.781s
| Adam | epoch: 005 | loss: 0.14793 - acc: 0.9594 | val_loss: 0.19385 - val_acc: 0.9403 -- iter: 1801/1801
--


## Avaliando o Modelo

In [13]:
model.evaluate(X_test,Y_test)

[0.9551569530782144]

Podemos salvar o modelo para utilizar no futuro

In [14]:
model.save('models/model_cnn_doc.tflearn')

INFO:tensorflow:/home/04449579445/workspaces/workspace_cognitiva/cursos_ia/pln_basico/models/model_cnn_doc.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## Realizando Predições com dados de Teste

Primeiro vamos carregá-lo. Note que esse passo não é necessário se já tivermos o modelo carregado em memória.

In [15]:
model.load('models/model_cnn_doc.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/04449579445/workspaces/workspace_cognitiva/cursos_ia/pln_basico/models/model_cnn_doc.tflearn


Então, para finalizar, vamos visualizar algumas predições com os dados de teste.

In [16]:
# Predição dos dados de testes
predictions = model.predict(X_test)

# Criação do dicionário reverso (key -> Índice ; value -> Token)
reverse_dictionary = dict(zip(tokenizer.word_index.values(), tokenizer.word_index.keys()))

# Varre os dados de teste e imprime a predição dos 5 primeiros textos
for i in range(len(X_test[:10])):
    documento='';
    for j in X_test[i]:
        if j != 0:
            documento+=reverse_dictionary[j]+' '
        else:
            break
    print('Texto ->',documento)
    print('    Label ->', Y_labels.columns[np.argmax(Y_test[i])])
    print('    Label predict->', Y_labels.columns[np.argmax(predictions[i])])    
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

Texto -> scotland gam ireland captain rul saturday six nat scotland origin nam start fail injury pick win ita replac nam train friday cent gordon also injury fit test friday see play would obvy replac cent could also mov could also ask travel squad scotland meas chang ireland sid see replac win third cap mak debut vict sou afric last novemb mil 
    Label -> sport
    Label predict-> sport
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Texto -> french hono direct park brit film direct sir al park mad off ord art let on frant cult hono sir al receiv par wednesday french cult min poss film tal mr said pres award park french film say hollywood cre us told min hono frant carry world sir al film includ commit found memb direct gre britain form chairm uk film council board brit film work campaign shown us art plac socy mr said show us link quest world work also direct 2003 film lif david play man row art commit sent 
    Label -

In [17]:
# Localizando no dataset as linhas que possuem um determinado texto
mask = np.column_stack([data[col].str.contains(r"mak green", na=False) for col in data])
data.loc[mask.any(axis=1)]

,Category,Text,Filename
985,tech,mak green comput hitech industry start get env...,307.txt


## Métricas de Classificação e Matriz de confusão

In [18]:
from sklearn import metrics

#Obtendo a categoria para as saídas dejedas e as previsões
ymax = np.argmax(Y_test,axis=1)
predictions_max= np.argmax(predictions,axis=1)

##Imprime as categorias
print("Categorias: ",Y_labels.columns)

##Imprime as métricas de classificação
print('Métricas de Classificação')
print(metrics.classification_report(ymax,predictions_max))

print('Matriz de Confusão')
print(metrics.confusion_matrix(ymax, predictions_max))

Categorias:  Index(['business', 'entertainment', 'politics', 'sport', 'tech'], dtype='object')
Métricas de Classificação
              precision    recall  f1-score   support

           0       0.98      0.96      0.97        55
           1       0.93      0.93      0.93        29
           2       0.98      0.89      0.93        45
           3       0.95      1.00      0.97        52
           4       0.93      0.98      0.95        42

   micro avg       0.96      0.96      0.96       223
   macro avg       0.95      0.95      0.95       223
weighted avg       0.96      0.96      0.95       223

Matriz de Confusão
[[53  0  0  0  2]
 [ 0 27  1  1  0]
 [ 1  1 40  2  1]
 [ 0  0  0 52  0]
 [ 0  1  0  0 41]]


---

# Exercício 

Modifique este jupyter para que o modelo obtenha um melhor desempenho.

Faça tentativas modificando as seguintes opções e compare os resultados:

- Uso dos datasets X_matrix_tfidf e X_matrix_count ao inves de X (lembre de mudar os passos de 'Separação dos dados' e 'Construção da Rede Neural Convolucional' comentando e descomentando as linhas indicadas)
- Uso de Lemmatization ao invés de Stemming
- Variação no número de features através da variável NUM_FEATURES
- Modificações dos hiperparâmetros da rede neural, tais como 'n_epoch' e 'batch_size'
- Modifique a arquitetura da rede neural, tal como a quantidade e tamanho dos filtros, quantidade de camadas etc

Qual configuração obteve melhores resultados?

---

# Fim